In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import glob
import acoustic_data_science.config as config

In [2]:
def loud_noises(df): 
    'function takes in broadband SPL dataframe and adds label True/False depending if time interval is classified as loud or not'
    'returns original dataframe with loud label and dataframe with just loud noises'
    
    # detection of transient events - example if mean noise of 'crack' is 10% louder than mean background noise 
    df['broadband_spl']>(df['background']+10)

    #new dataframe of transient events for plotting 
    loud_noise = df[df['loud']==True]
    loud_noise.reset_index(inplace=True)
    
    #prints number of loud noise events as number and percentage of overall time 
    print('Number of loud noise events detected {}, {:.2}% of total time'.format(len(loud_noise), len(loud_noise)/len(df)*100))

    return df, loud_noise

In [3]:
def transients_from_loud(df):
    'FASTER METHOD - inserts new column (transient) with a True / False flag for a transient event '
    
    df['trans_shft_down'] = np.concatenate((np.array([True]), df['loud'][:-1]))
    df['trans_shft_up'] = np.concatenate((df['loud'][1:], np.array([True])))
    df['transient'] = df['loud'] & (df['trans_shft_down']==False) & (df['trans_shft_up']==False)
    df.drop(columns=['trans_shft_down', 'trans_shft_up'], inplace=True)
    
    return df

In [5]:
feather_folder_path = config.interim_data_path
feather_paths = sorted(glob.glob(f'{feather_folder_path}/*.feather'))
for feather_path in feather_paths:
    print(feather_path)
    
    df = pd.read_feather(path=feather_path)
    df = df.dropna(axis='rows', subset=['background']).reset_index()

    # Tag loud noises.
    df['loud'] = df['broadband_spl']>(df['background']+10)
    
    # Tag cracks.
    df = transients_from_loud(df)

    #df.to_feather(path=f"{config.processed_data_path}/{feather_path.split('/')[-1]}")

../data/interim/2018_08.feather
../data/interim/2018_09.feather
../data/interim/2018_10.feather
../data/interim/2018_11.feather
../data/interim/2018_12.feather
../data/interim/2019_01.feather
../data/interim/2019_02.feather
../data/interim/2019_03.feather
../data/interim/2019_04.feather
../data/interim/2019_05.feather
